# Plots for the paper

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gwpy.timeseries import TimeSeries

%matplotlib inline

## TSeries figure

### Load data

In [ ]:
fs = 512  # sampling rate

def load_data(channels):
    
    dset = []
    norm_factors = []
    
    for channel in channels:
        
        # load data
        fname = f'data/{channel}' 
        data = np.load(fname)
        
        # remove sides due to whitening artifacts
        gps_times = data[fs*4:-fs*4, 0]
        data_tseries = data[fs*4:-fs*4, 1].reshape(-1, 1)
        
        # normalize and append tseries
        norm_factor = np.max(np.abs(data_tseries))
        norm_factors.append(norm_factor)
        dset.append(data_tseries / norm_factor)
    
    dset = np.squeeze(dset)
    dset = np.float32(dset)
    gps_times = np.array(gps_times)
    
    return dset, gps_times, norm_factors


# load tseries data
channels = ['DCS-CALIB_STRAIN_CLEAN_C01_512Hz_event_4096s_whitened.npy',
            'LSC-POP_A_RF45_I_ERR_DQ_512Hz_event_4096s_whitened.npy',
            'LSC-POP_A_RF45_Q_ERR_DQ_512Hz_event_4096s_whitened.npy',
            'LSC-POP_A_RF9_I_ERR_DQ_512Hz_event_4096s_whitened.npy']

dset, gps_times, norm_factors = load_data(channels)

### Make the plot

In [ ]:
t0 = 1264316116.435
event_idx = np.abs(gps_times - t0).argmin()

dur = 6
time = np.linspace(0, dur, fs*dur) - 2

dx1 = int(fs*2)
dx2 = int(fs*4)
dset0_norm = dset[0,event_idx-dx1:event_idx+dx2]/np.max(np.abs(dset[0,event_idx-dx1:event_idx+dx2]))
dset1_norm = dset[1,event_idx-dx1:event_idx+dx2]/np.max(np.abs(dset[1,event_idx-dx1:event_idx+dx2]))
dset2_norm = dset[2,event_idx-dx1:event_idx+dx2]/np.max(np.abs(dset[2,event_idx-dx1:event_idx+dx2]))
dset3_norm = dset[3,event_idx-dx1:event_idx+dx2]/np.max(np.abs(dset[3,event_idx-dx1:event_idx+dx2]))

In [ ]:
plt.figure(figsize=(6,10))
plt.subplot(4,1,1)
plt.plot(time, dset0_norm, label= 'h(t)')
plt.xlim([-2,4])
plt.legend(loc='upper left', fontsize=14)
plt.subplot(4,1,2)
plt.plot(time, dset1_norm, label= 'RF45-Q')
plt.xlim([-2,4])
plt.legend(loc='upper left', fontsize=14)
plt.subplot(4,1,3)
plt.plot(time, dset2_norm, label= 'RF45-I')
plt.xlim([-2,4])
plt.legend(loc='upper left', fontsize=14)
plt.subplot(4,1,4)
plt.plot(time, dset3_norm, label= 'RF9-I')
plt.xlim([-2,4])
plt.xlabel('Time (seconds) from 2020-01-29 06:54:58.435 UTC', fontsize=14)
plt.legend(loc='upper left', fontsize=14)
plt.tight_layout()
plt.savefig(f'output/plots/paper_tseries.png', dpi=400)

## Spectrogram

In [ ]:
data = np.load(f'data/DCS-CALIB_STRAIN_CLEAN_C01_4096Hz_event_4096s.npy')
orig_tseries = TimeSeries(data[:,1], times=data[:,0] - t0)

data = np.load(f'output/DCS-CALIB_STRAIN_CLEAN_C01_4096Hz_event_cleaned.npy')
cleaned_tseries = TimeSeries(data[1,:], times=data[0,:] - t0)

In [ ]:
# plotting params
crop_win = 20
plot_win = 2

# crop data for faster q transforms
orig_tseries_cropped = orig_tseries.crop(-crop_win,crop_win)
cleaned_tseries_cropped = cleaned_tseries.crop(-crop_win, crop_win)

dataset = ['orig','clean','diff']
q_trans = {}
q_trans['orig'] = orig_tseries_cropped.q_transform(outseg=(-plot_win, plot_win),qrange=(10,20))
q_trans['clean'] = cleaned_tseries_cropped.q_transform(outseg=(-plot_win, plot_win),qrange=(10,20))
q_trans['diff'] = q_trans['orig'] - q_trans['clean']

ylim = (10, 512) 
alim = (0, 25)

label = {}
label['orig'] = 'Original data'
label['clean'] = 'Cleaned data'
label['diff'] = 'Original - Cleaned'

plot, axes = plt.subplots(nrows=3, sharex=True, figsize=(3.375*2.0,3.375*3.0))

for i, ax in zip(dataset,axes):

    pcm = ax.imshow(q_trans[i], vmin=alim[0], vmax=alim[1])
    ax.set_ylim(ylim[0], ylim[1])
    ax.set_xlabel('')
    ax.set_yscale('log')
    ax.plot([0], 10, label=label[i], visible=False)
    ax.grid(alpha=0.6)
    ax.legend(loc='upper left', handlelength=0, handletextpad=0)

axes[1].set_ylabel(r"$\mathrm{Frequency \ (Hz)}$")
axes[-1].set_xlabel(r"$\mathrm{Time \ (seconds)}$")
cbar = axes[0].colorbar(clim=(alim[0], alim[1]),location='top')
cbar.set_label(r"$\mathrm{Normalized \ energy}$");

plot.tight_layout()
plot.subplots_adjust(top=0.85)
plot.savefig('output/plots/paper_oscan.png', dpi=400)

## Stats plot
do this on Stats folder/env